# Tutorial 2 - Hierarchical Agents

## Functionalities:
- An agent (referred as meta agent) can also assign other agents (referred as inner agents) as the function call, thereby creating a hierarchical structure
- Whenever the inner agent is called, we will run the full inner agent's process until termination
- Maximum number of steps for each agent (meta agent or inner agent) can be defined in `max_subtasks` (default: 5) variable when initialising agent

## Additional Details:
- The inner agent will take in meta agent's instruction as input
- In addition, the inner agent will also have access to meta agent's task and subtasks completed for global context
- Inner agent's output will directly go into meta agent's subtasks completed dictionary, enabling it to directly update global context

## Design Philosophy:
- Bottom layers can get context of the top layers for more context-specific generation
- Top layers choose bottom layers in a hierarchical fashion so as to limit the number of choices at each level for better performance
- `subtasks_completed` serves as the global context for the meta agents and the inner agents, and can be summarised as needed, e.g. when performing a new task to filter out irrelevant information 

# Install TaskGen

In [1]:
# !pip install taskgen-ai

In [2]:
# Set up API key and do the necessary imports
import os
from taskgen import *

os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

# Define Agents and Meta Agents
- You can also create a Meta Agent that uses other Agents (referred to as Inner Agents) as functions
- Create your Meta agent using `Agent()` (Note: No different from usual process of creating Agents - your Meta Agent is also an Agent)
- Set up an Inner Agent list and assign it to your Meta agent using `assign_agents(agent_list)`

## Example Meta Agent Setup
```python
# Define your meta-agent
my_agent = Agent('Menu Creator', 
                 'Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.')

# Define your agent list. Note you can just assign functions to the agent in place using .assign_functions(function_list)
agent_list = [
    Agent('Chef', 'Takes in dish names and generates ingredients for each of them. Does not generate prices.'),
    Agent('Boss', 'Takes in menu items and curates them based on profitability'),
    Agent('Creative Writer', 'Takes in a cuisine type and generates interesting dish names and descriptions.', max_subtasks = 2),
    Agent('Economist', 'Takes in dish names and comes up with fictitious pricing for each of them')
    ]

my_agent.assign_agents(agent_list)
```

## Run the Meta Agent
- Let us run the agent and see the interactions between the Meta Agent and Inner Agents to solve the task!
```python
output = my_agent.run('Give me 5 menu items with name, description, ingredients and price based on Italian food choices.')
```

In [3]:
def random_number_from_string(input_string):
    ''' Returns a random number between 1 and 10 when given an input string '''
    import hashlib
    import random
    # Hash the input string
    hashed_string = hashlib.sha256(input_string.encode()).hexdigest()
    
    # Convert the hashed string to an integer
    hashed_integer = int(hashed_string, 16)
    
    # Seed the random number generator with the hashed integer
    random.seed(hashed_integer)
    
    # Generate a random number between 1 and 10
    return random.randint(1, 10)

In [4]:
# assign a dish pricing function for economist
# Note: Try to make the output informative that is understandable by the Agent - use free text or json to explain the output so that someone can understand without the input
def dish_price(list_of_dish_names: list) -> dict:
    ''' Takes in <list_of_dish_names> and outputs price '''
    if not isinstance(list_of_dish_names, list):
        list_of_food_items = [list_of_dish_names]
    output = {}
    for each in list_of_dish_names:
        output[each] = '$'+str(random_number_from_string(each))
    return output
    
fp = Function(fn_description = "Takes in <list_of_dish_names: list> and outputs price", 
              output_format = {"Price": "Dish name as key and price as value"}, 
              examples = {"list_of_dish_names": ["Pasta", "Pizza"], "Price": [{'Pasta': '$5'}, {'Pizza':'$2'}]},
              external_fn = dish_price)

In [5]:
# Define your meta-agent
my_agent = Agent('Menu Creator', 
                 'Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.')

In [6]:
# Define your agent list. Note you can just assign functions to the agent in place using .assign_functions(function_list)
agent_list = [
    Agent('Chef', 'Takes in dish names and generates ingredients for each of them. Does not generate prices.'),
    Agent('Boss', 'Takes in menu items and curates them based on profitability'),
    Agent('Creative Writer', 'Takes in a cuisine type and generates interesting dish names and descriptions. Does not generate prices or ingredients.', max_subtasks = 2),
    Agent('Economist', 'Takes in dish names and comes up with fictitious pricing for each of them').assign_functions([fp])
    ]

In [7]:
my_agent.assign_agents(agent_list)

# Approach 1: Run a generic task with the Meta-Agent
- Let us run a general task to create 5 menu items for an Italian restaurant
- You can also specify how Agents interact with one another in the task, otherwise the Meta-Agent will infer what should be done

In [8]:
output = my_agent.run('Give me 5 menu items with name, description, ingredients and price based on Italian food choices.')

Subtask identified: Generate interesting dish names and descriptions for Italian cuisine
Calling function Creative Writer with parameters {'instruction': 'Generate interesting dish names and descriptions for Italian cuisine'}

### Start of Inner Agent: Creative Writer ###
Subtask identified: Research popular Italian dishes
Getting LLM to perform the following task: Research popular Italian dishes
> {
1. "Name": "Ravioli di Zucca",
   "Description": "Homemade ravioli filled with roasted pumpkin, ricotta cheese, and sage butter sauce.",
   "Ingredients": "Pumpkin, ricotta cheese, sage, butter, flour, eggs",
   "Price": "N/A"
2. "Name": "Osso Buco",
   "Description": "Braised veal shanks served with gremolata and risotto alla Milanese.",
   "Ingredients": "Veal shanks, gremolata (lemon zest, garlic, parsley), Arborio rice, saffron",
   "Price": "N/A"
3. "Name": "Tiramisu",
   "Description": "Classic Italian dessert made with layers of coffee-soaked ladyfingers and mascarpone cream, dusted

In [9]:
output = my_agent.reply_user()

{'1': {'Name': 'Gnocchi al Tartufo', 'Description': 'Handmade potato dumplings in a luxurious truffle cream sauce.', 'Ingredients': 'Potatoes, truffle cream, flour, eggs', 'Price': '$6'}, '2': {'Name': 'Pollo Marsala', 'Description': 'Tender chicken breast cooked in a Marsala wine sauce with mushrooms.', 'Ingredients': 'Chicken breast, Marsala wine, mushrooms, garlic', 'Price': '$5'}, '3': {'Name': 'Cannoli Siciliani', 'Description': 'Crispy pastry tubes filled with sweet ricotta cheese and chocolate chips.', 'Ingredients': 'Pastry dough, ricotta cheese, chocolate chips, sugar', 'Price': '$9'}, '4': {'Name': 'Linguine alle Vongole', 'Description': 'Linguine pasta with fresh clams in a white wine garlic sauce.', 'Ingredients': 'Linguine pasta, clams, white wine, garlic', 'Price': '$2'}, '5': {'Name': 'Saltimbocca alla Romana', 'Description': 'Thin veal cutlets wrapped in prosciutto and sage, served with a white wine sauce.', 'Ingredients': 'Veal cutlets, prosciutto, sage, white wine', '

In [10]:
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Give me 5 menu items with name, description, ingredients and price based on Italian food choices.
Subtasks Completed:
Subtask: Research popular Italian dishes
{
1. "Name": "Ravioli di Zucca",
   "Description": "Homemade ravioli filled with roasted pumpkin, ricotta cheese, and sage butter sauce.",
   "Ingredients": "Pumpkin, ricotta cheese, sage, butter, flour, eggs",
   "Price": "N/A"
2. "Name": "Osso Buco",
   "Description": "Braised veal shanks served with gremolata and risotto alla Milanese.",
   "Ingredients": "Veal shanks, gremolata (lemon zest, garlic, parsley), Arborio rice, saffron",
   "Price": "N/A"
3. "Name": "Tiramisu",
   "Description": "Classic Italian dessert made with layers of coffee-soaked ladyfingers and mascarpone cream, dusted with c

# Approach 2: Step through the process yourself as the Meta Agent
- You can use agents manually using `use_agent(agent_name: str, agent_task: str)` The outputs of the agents will be stored into `subtasks_completed` of the meta agent automatically
- You can also interact with the meta agent using `reply_user(task: str, stateful: bool = False)`. This generates a reply for the agent to the `task` using `subtasks_completed` as context. If `stateful` is true, this will also store the output into `subtasks_completed`
- If `subtasks_completed` is too lengthy, you can run `summarise_subtasks_completed(task: str)` This summarises `subtasks_completed` according to what is relevant to the `task`
- If you want to redo a subtask, you can remove the most recent subtask from `subtasks_completed` using `remove_last_subtask()`

In [11]:
# reset the state of the agent
my_agent.reset()
# currently should have no task and nothing in subtasks_completed
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: No task assigned
Subtasks Completed: None
Is Task Completed: False


In [12]:
# Assign a task for the meta agent - this helps to contextualise what you give each of the inner agents later on
my_agent.assign_task('Give me 5 menu items with name, description, ingredients and price based on Italian food choices.')

In [13]:
# This will run the Agent Creative Writer (with shared subtasks_completed as meta agent)
# Output of task will go directly into subtasks_completed - we view the inner agent as simply an extension of the meta agent
my_agent.use_agent('Creative Writer', 'Generate 5 Italian dish names and descriptions')

Calling function Creative Writer with parameters {'instruction': 'Generate 5 Italian dish names and descriptions'}

### Start of Inner Agent: Creative Writer ###
Subtask identified: Brainstorm and create a unique name and description for the first Italian dish
Getting LLM to perform the following task: Brainstorm and create a unique name and description for the first Italian dish
> Name: Truffle Risotto Delight Description: Indulge in a creamy risotto infused with earthy truffle flavors, topped with Parmesan shavings for a luxurious touch.

Subtask identified: Brainstorm and create a unique name and description for the second Italian dish
Getting LLM to perform the following task: Brainstorm and create a unique name and description for the second Italian dish
> Name: Tuscan Sun Pasta Description: Transport yourself to the sunny hills of Tuscany with this vibrant pasta dish featuring sun-dried tomatoes, olives, and fresh basil in a light garlic-infused olive oil sauce.

### End of Inner

In [14]:
# subtasks_completed should be updated with the Creative Agent's outputs
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Give me 5 menu items with name, description, ingredients and price based on Italian food choices.
Subtasks Completed:
Subtask: Brainstorm and create a unique name and description for the first Italian dish
Name: Truffle Risotto Delight Description: Indulge in a creamy risotto infused with earthy truffle flavors, topped with Parmesan shavings for a luxurious touch.

Subtask: Brainstorm and create a unique name and description for the second Italian dish
Name: Tuscan Sun Pasta Description: Transport yourself to the sunny hills of Tuscany with this vibrant pasta dish featuring sun-dried tomatoes, olives, and fresh basil in a light garlic-infused olive oil sauce.

Subtask: Generate 5 Italian dish names and descriptions
{'Status': 'Completed'}

Is Task Comple

In [15]:
my_agent.use_agent('Chef', 'Generate ingredients for the 5 Italian dishes')

Calling function Chef with parameters {'instruction': 'Generate ingredients for the 5 Italian dishes'}

### Start of Inner Agent: Chef ###
Subtask identified: Brainstorm and create a unique name and description for the third Italian dish
Getting LLM to perform the following task: Brainstorm and create a unique name and description for the third Italian dish
> Name: Caprese Chicken Delight Description: Savor the flavors of Italy with this succulent grilled chicken topped with ripe tomatoes, fresh basil, and creamy mozzarella, drizzled with balsamic glaze.

Subtask identified: Brainstorm and create a unique name and description for the fifth Italian dish
Getting LLM to perform the following task: Brainstorm and create a unique name and description for the fifth Italian dish
> Name: Sicilian Seafood Feast Description: Dive into a medley of fresh seafood including shrimp, clams, and mussels, simmered in a savory tomato broth with garlic, herbs, and a hint of white wine.

Subtask identified

In [16]:
my_agent.use_agent('Economist', 'Generate prices for the 5 Italian dishes')

Calling function Economist with parameters {'instruction': 'Generate prices for the 5 Italian dishes'}

### Start of Inner Agent: Economist ###
Subtask identified: Use dish_price function to generate prices for the 5 Italian dishes
Calling function dish_price with parameters {'list_of_dish_names': ['Truffle Risotto Delight', 'Tuscan Sun Pasta', 'Caprese Chicken Delight', 'Sicilian Seafood Feast', 'Fifth Italian Dish']}
> {'Price': {'Truffle Risotto Delight': '$8', 'Tuscan Sun Pasta': '$1', 'Caprese Chicken Delight': '$5', 'Sicilian Seafood Feast': '$10', 'Fifth Italian Dish': '$8'}}

Task completed successfully!

### End of Inner Agent: Economist ###

> {'Status': 'Completed'}



In [17]:
my_agent.use_agent('Boss', 'Choose only 3 Italian dishes to serve that will please Gordon Ramsay and explain why')

Calling function Boss with parameters {'instruction': 'Choose only 3 Italian dishes to serve that will please Gordon Ramsay and explain why'}

### Start of Inner Agent: Boss ###
Subtask identified: Choose 3 Italian dishes that will please Gordon Ramsay
Getting LLM to perform the following task: Choose 3 Italian dishes that will please Gordon Ramsay
> Based on profitability, the 3 Italian dishes that will please Gordon Ramsay are Truffle Risotto Delight, Caprese Chicken Delight, and Sicilian Seafood Feast.

Subtask identified: Explain why each dish will please Gordon Ramsay
Getting LLM to perform the following task: Explain why each dish will please Gordon Ramsay
> Truffle Risotto Delight, Caprese Chicken Delight, and Sicilian Seafood Feast will please Gordon Ramsay due to their profitability and high-quality ingredients, such as truffle oil, fresh basil, and seafood, which align with his standards for flavorful and well-executed dishes.

Task completed successfully!

### End of Inner A

In [18]:
# Get a targeted response from your agent, and add this to subtask_completed
output = my_agent.reply_user('Choose the 3 menu items for Gordon Ramsay line by line in this format - Dish Name, Dish Description, Dish Ingredients, Dish Price')

Truffle Risotto Delight, Name: Truffle Risotto Delight Description: Indulge in a creamy risotto infused with earthy truffle flavors, topped with Parmesan shavings for a luxurious touch. Ingredients: Arborio rice, truffle oil, chicken broth, Parmesan cheese Price: $8
Caprese Chicken Delight, Name: Caprese Chicken Delight Description: Savor the flavors of Italy with this succulent grilled chicken topped with ripe tomatoes, fresh basil, and creamy mozzarella, drizzled with balsamic glaze. Ingredients: Chicken breast, tomatoes, fresh basil, mozzarella cheese, balsamic glaze Price: $5
Sicilian Seafood Feast, Name: Sicilian Seafood Feast Description: Dive into a medley of fresh seafood including shrimp, clams, and mussels, simmered in a savory tomato broth with garlic, herbs, and a hint of white wine. Ingredients: Shrimp, clams, mussels, tomato broth, garlic, herbs, white wine Price: $10


In [19]:
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Give me 5 menu items with name, description, ingredients and price based on Italian food choices.
Subtasks Completed:
Subtask: Brainstorm and create a unique name and description for the first Italian dish
Name: Truffle Risotto Delight Description: Indulge in a creamy risotto infused with earthy truffle flavors, topped with Parmesan shavings for a luxurious touch.

Subtask: Brainstorm and create a unique name and description for the second Italian dish
Name: Tuscan Sun Pasta Description: Transport yourself to the sunny hills of Tuscany with this vibrant pasta dish featuring sun-dried tomatoes, olives, and fresh basil in a light garlic-infused olive oil sauce.

Subtask: Generate 5 Italian dish names and descriptions
{'Status': 'Completed'}

Subtask: Brain

In [20]:
# you can summarise the subtasks_history for the next task
my_agent.summarise_subtasks_completed('Generate the 3 Italian dishes for Gordon Ramsay in menu format')

Truffle Risotto Delight, Name: Truffle Risotto Delight Description: Indulge in a creamy risotto infused with earthy truffle flavors, topped with Parmesan shavings for a luxurious touch. Ingredients: Arborio rice, truffle oil, chicken broth, Parmesan cheese Price: $8
Caprese Chicken Delight, Name: Caprese Chicken Delight Description: Savor the flavors of Italy with this succulent grilled chicken topped with ripe tomatoes, fresh basil, and creamy mozzarella, drizzled with balsamic glaze. Ingredients: Chicken breast, tomatoes, fresh basil, mozzarella cheese, balsamic glaze Price: $5
Sicilian Seafood Feast, Name: Sicilian Seafood Feast Description: Dive into a medley of fresh seafood including shrimp, clams, and mussels, simmered in a savory tomato broth with garlic, herbs, and a hint of white wine. Ingredients: Shrimp, clams, mussels, tomato broth, garlic, herbs, white wine Price: $10


In [21]:
# subtask has been summarised, helping to reduce the context significantly for later task generation
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Give me 5 menu items with name, description, ingredients and price based on Italian food choices.
Subtasks Completed:
Subtask: Summary of Generate the 3 Italian dishes for Gordon Ramsay in menu format
Truffle Risotto Delight, Name: Truffle Risotto Delight Description: Indulge in a creamy risotto infused with earthy truffle flavors, topped with Parmesan shavings for a luxurious touch. Ingredients: Arborio rice, truffle oil, chicken broth, Parmesan cheese Price: $8
Caprese Chicken Delight, Name: Caprese Chicken Delight Description: Savor the flavors of Italy with this succulent grilled chicken topped with ripe tomatoes, fresh basil, and creamy mozzarella, drizzled with balsamic glaze. Ingredients: Chicken breast, tomatoes, fresh basil, mozzarella cheese, b

In [22]:
# we also allow you to manually remove the last item added to subtasks_completed, so you can revert your agent state if needed for backtracking
my_agent.remove_last_subtask()

Removed last subtask from subtasks_completed: ('Summary of Generate the 3 Italian dishes for Gordon Ramsay in menu format', 'Truffle Risotto Delight, Name: Truffle Risotto Delight Description: Indulge in a creamy risotto infused with earthy truffle flavors, topped with Parmesan shavings for a luxurious touch. Ingredients: Arborio rice, truffle oil, chicken broth, Parmesan cheese Price: $8\nCaprese Chicken Delight, Name: Caprese Chicken Delight Description: Savor the flavors of Italy with this succulent grilled chicken topped with ripe tomatoes, fresh basil, and creamy mozzarella, drizzled with balsamic glaze. Ingredients: Chicken breast, tomatoes, fresh basil, mozzarella cheese, balsamic glaze Price: $5\nSicilian Seafood Feast, Name: Sicilian Seafood Feast Description: Dive into a medley of fresh seafood including shrimp, clams, and mussels, simmered in a savory tomato broth with garlic, herbs, and a hint of white wine. Ingredients: Shrimp, clams, mussels, tomato broth, garlic, herbs, 

In [23]:
# visualise your agent's status - should have empty subtasks_completed
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Give me 5 menu items with name, description, ingredients and price based on Italian food choices.
Subtasks Completed: None
Is Task Completed: False
